In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
import __init__
#import models.random as random_model
#import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel
import models.outputmaker as outputmaker
import evaluator.metrics as metrics

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


In [6]:
import ast
import csv
import random
import statistics
import sys
import sklearn

def spans_to_ents(doc, spans, label):
    """Converts span indicies into spacy entity labels."""
    started = False
    left, right, ents = 0, 0, []
    for x in doc:
        if x.pos_ == 'SPACE':
            continue
        if spans.intersection(set(range(x.idx, x.idx + len(x.text)))):
            if not started:
                left, started = x.idx, True
            right = x.idx + len(x.text)
        elif started:
            ents.append((left, right, label))
            started = False
    if started:
        ents.append((left, right, label))
    return ents

"""Automatic best effort span cleaner for SemEval 2021 Toxic Spans."""

import ast
import csv
import itertools
import string
import sys

SPECIAL_CHARACTERS = string.whitespace

def _contiguous_ranges(span_list):
    """Extracts continguous runs [1, 2, 3, 5, 6, 7] -> [(1,3), (5,7)]."""
    output = []
    for _, span in itertools.groupby(
        enumerate(span_list), lambda p: p[1] - p[0]):
        span = list(span)
        output.append((span[0][1], span[-1][1]))
    return output


def fix_spans(spans, text, special_characters=SPECIAL_CHARACTERS):
    """Applies minor edits to trim spans and remove singletons."""
    cleaned = []
    for begin, end in _contiguous_ranges(spans):
        while text[begin] in special_characters and begin < end:
            begin += 1
        while text[end] in special_characters and begin < end:
            end -= 1
        if end - begin > 1:
            cleaned.extend(range(begin, end + 1))
    return cleaned



In [17]:
index = 100
print("Spans:", train.iloc[index].spans)
print("Text :", train.iloc[index].text)

Spans: [22, 23, 24, 127, 128, 129, 130, 131, 132, 133, 134, 135]
Text : My neighborhood had a gay, straight, tranny, granny, psycho, non-psycho, maybe=psycho, could be psycho
march for under noticed imbeciles.....

everyone showed up


In [18]:
spans = train.iloc[index].spans
text = train.iloc[index].text

In [19]:
spans = fix_spans(spans, text)

In [20]:
spans_to_ents(nlp(text), set(spans), label='toxic')

[(22, 25, 'toxic'), (127, 136, 'toxic')]

In [21]:
func = lambda x:''.join([' '+x[0][x[1][i]] if x[1][i-1] + 1 != x[1][i] else x[0][x[1][i]] 
                                      for i in range(len(x[1]))])[1:]

In [57]:
toxic_words = func([text, spans])
nlp_text = nlp(text)
cns_ranges = _contiguous_ranges(spans)
#ents = spans_to_ents(nlp(text), set(spans), label='toxic')

In [58]:
ents, toxic_words.split(), cns_ranges

([(22, 25, 'toxic'), (127, 136, 'toxic')],
 ['gay', 'imbeciles'],
 [(22, 24), (127, 135)])

In [59]:
def check_toxicity(word, text, ent_ranges):
    print(text.index(word), word, ent_ranges)
    index = text.index(word)
    for ent_range in ent_ranges:
        if index >= ent_range[0] and index <= ent_range[1]:
            return True
    return False

In [60]:
X = []
for word in nlp_text:
    if (word.text in toxic_words) and check_toxicity(word.text, text, cns_ranges):
        X.append((word.text, word.pos_, 'toxic'))
    else:
        X.append((word.text, word.pos_, 'normal'))
    
#return X, spans, toxic_words, text

17 a [(22, 24), (127, 135)]
22 gay [(22, 24), (127, 135)]
76 be [(22, 24), (127, 135)]
127 imbeciles [(22, 24), (127, 135)]


In [61]:
X

[('My', 'DET', 'normal'),
 ('neighborhood', 'NOUN', 'normal'),
 ('had', 'AUX', 'normal'),
 ('a', 'DET', 'normal'),
 ('gay', 'ADJ', 'toxic'),
 (',', 'PUNCT', 'normal'),
 ('straight', 'ADJ', 'normal'),
 (',', 'PUNCT', 'normal'),
 ('tranny', 'PROPN', 'normal'),
 (',', 'PUNCT', 'normal'),
 ('granny', 'NOUN', 'normal'),
 (',', 'PUNCT', 'normal'),
 ('psycho', 'NOUN', 'normal'),
 (',', 'PUNCT', 'normal'),
 ('non', 'ADJ', 'normal'),
 ('-', 'ADJ', 'normal'),
 ('psycho', 'ADJ', 'normal'),
 (',', 'PUNCT', 'normal'),
 ('maybe', 'ADV', 'normal'),
 ('=', 'ADP', 'normal'),
 ('psycho', 'NOUN', 'normal'),
 (',', 'PUNCT', 'normal'),
 ('could', 'VERB', 'normal'),
 ('be', 'AUX', 'normal'),
 ('psycho', 'PROPN', 'normal'),
 ('\n', 'SPACE', 'normal'),
 ('march', 'PROPN', 'normal'),
 ('for', 'ADP', 'normal'),
 ('under', 'ADP', 'normal'),
 ('noticed', 'ADJ', 'normal'),
 ('imbeciles', 'NOUN', 'toxic'),
 ('.....', 'PUNCT', 'normal'),
 ('\n\n', 'SPACE', 'normal'),
 ('everyone', 'PRON', 'normal'),
 ('showed', 'VER

In [62]:
!pip3 install hmmlearn

Error initializing plugin EntryPoint(name='Windows (alt)', value='keyrings.alt.Windows', group='keyring.backends').
Traceback (most recent call last):
  File "/home/hb/.local/lib/python3.6/site-packages/keyring/backend.py", line 203, in _load_plugins
    init_func = ep.load()
  File "/home/hb/.local/lib/python3.6/site-packages/importlib_metadata/__init__.py", line 105, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_remov